In [1]:
# Librerías necesarias
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np

In [2]:
# Función para extraer los valores medidos del voltaje y la corriente del archivo .ldf correspondiente

def extract_data_from_ldf(root_extract):

    data_i = []
    data_v = []
    data_std_i = []
    data_std_v = []

    aux_1 = 0
    aux_2 = 0
    aux_3 = 0
    aux_4 = 0
    find = "class=\"correctedData\">"
    
    with open(root_extract, 'r', encoding='utf-8') as infile:
        for line in infile:
            if aux_1==1:
                if aux_2==1:    
                    if aux_3==1: 
                        if aux_4==1:    
                            line = line.lstrip("            <vec label=\"stdV\">;")
                            line=line[:-8]
                            data_std_v.append(line)
                            break   
                        line = line.lstrip("            <vec label=\"stdI\">;")
                        line=line[:-8]
                        data_std_i.append(line)
                        aux_4 = 1 
                    if aux_3 == 0:
                        line = line.lstrip("            <vec label=\"V\">;")
                        line=line[:-8]
                        data_v.append(line)
                        aux_3 = 1
                if aux_2 == 0:
                    line = line.lstrip("            <vec label=\"I\">;")
                    line=line[:-8]
                    aux_2 = 1
                    data_i.append(line)
            if aux_1 == 0:
                line = line.rstrip().split(" ")
                if find in line:
                    aux_1 = 1
    datai_ = [float(i) for i in data_i[0].split(";")]
    datav_ = [float(v) for v in data_v[0].split(";")]
    datastdi_ = [float(stdi) for stdi in data_std_i[0].split(";")]
    datastdv =  [float(stdv) for stdv in data_std_v[0].split(";")]

    return [datai_, datav_, datastdi_, datastdv]

In [3]:
# Función para extraer los parámetros del archivo .ldf correspondiente 

def extract_parameters_from_ldf(root_extract):

    def extract_number(line):

        i_0 = 0
        i_f = 0
        line_aux = line[13:-1]
        for i in range(len(line_aux)):
            if line_aux[i] =='>':
                i_0 = i
            if line_aux[i] == '<':
                i_f = i
        if line_aux[i_0+1:i_f] == "nan":
            number = "nan"
        else:
            number = float(line_aux[i_0+1:i_f])
        return number
    
    def extract_name(line):

        i_0 = 0
        i_f = 0
        line_aux = line[12:]
        for i in range(len(line_aux)):
            if line_aux[i] =='<':
                i_0 = i
            if line_aux[i] == '>':
                i_f = i
                break
        name = line_aux[i_0+1:i_f]
        return name

    parameters_name = []
    parameters_unit = ["[A/m^2]","[A/m^2]","[µm]","[m^-3]","[m^-3]","[V]","[V]","[V]","[V]","[eV]","[m^-3]","[eV]","[mA]","[eV]"]
    parameters_value = []
    parameters_error = []
    aux_1 = 0
    i=0

    find = "            <Je"
    with open(root_extract, 'r', encoding='utf-8') as infile:
        for line in infile:
            if i == 14:
                break
            if aux_1 == 0:
                line_aux = line.rstrip().split(">")
                if find in line_aux:
                    parameters_name.append(extract_name(line.rstrip("\n")))
                    parameters_value.append(extract_number(line.rstrip("\n")))
                    aux_1 = 1
            elif aux_1 == 1:
                parameters_error.append(extract_number(line.rstrip("\n")))
                i += 1
                aux_1 = 2
            else:
                parameters_name.append(extract_name(line.rstrip("\n")))
                parameters_value.append(extract_number(line.rstrip("\n")))
                aux_1 = 1

    parameters_value = [float(pv) for pv in parameters_value]

    return [parameters_name, parameters_unit, parameters_value, parameters_error]

In [4]:
def extract_settings_from_ldf(root_extract):
                                                             
    setting_name_list = []
    setting_value_list = []
    with open(root_extract, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()
        for i in range(20): # 20: total settings
            line = lines[89+i].split("<")
            line = ''.join(line).split(">")
            setting_name = line[0].strip()
            setting_name_list.append(setting_name)
            
            if line[1].split("/")[0] !='nan':
                setting_value = float(line[1].split("/")[0])
                setting_value_list.append(setting_value)
                
            else:
                setting_value = line[1].split("/")[0]
                setting_value_list.append(setting_value)
                
    return [setting_name_list, setting_value_list]
            

In [5]:
def extract_extradata_from_ldf(root_extract):
    
    extra_data_list = []
    extra_value_list = []
    
    with open(root_extract, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()
        for i in range(3): # 3: total extra data
            line = lines[4+i].split("<")
            line = ''.join(line).split(">")
            extra_name = line[0].strip()
            extra_data_list.append(extra_name)
            
            if line[1].split("/")[0].isdigit():
                extra_value = float(line[1].split("/")[0])
                extra_value_list.append(extra_value)
                
            else:
                extra_value = line[1].split("/")[0].strip()
                extra_value_list.append(extra_value)
                
    extra_information = ['Name file', 'Description', 'Comment']
    extra_value_information = ['name file here', 'Some description here', 'some coment here']
    extra_data_list = extra_data_list + extra_information
    extra_value_list = extra_value_list + extra_value_information
    
    return [extra_data_list, extra_value_list]

## -----------------------------------------------------------------

In [6]:
## Código final (engloba todas la funciones anteriores)

def return_all_data(root_extract):

    data_iv_ = extract_data_from_ldf(root_extract)
    data_parameters_ = extract_parameters_from_ldf(root_extract)
    data_settings_ = extract_settings_from_ldf(root_extract)
    data_extra_ = extract_extradata_from_ldf(root_extract)
    
    data_IV = pd.DataFrame({'Voltage': data_iv_[0], 
                            'Current': data_iv_[1], 
                            'std Voltage': data_iv_[2], 
                            'std Current': data_iv_[3]})
    
    data_parameters = pd.DataFrame({'Parameters': data_parameters_[0], 
                                    'Unit': data_parameters_[1], 
                                    'Value': data_parameters_[2], 
                                    'Error': data_parameters_[3]})
    
    data_settings = pd.DataFrame({'Setting': data_settings_[0], 
                                'Value': data_settings_[1]})
    
    data_extra = pd.DataFrame({'Extra data': data_extra_[0], 
                               'Value': data_extra_[1]})
    
    return [data_IV, data_parameters, data_settings, data_extra]

In [9]:
root = "../sputtering/SS-S-280423-Al-2E(-6)mbar-MR-C/Variando Potencia -10E-2mbar cte/20W.ldf"
all_data = return_all_data(root)

## -----------------------------------------------------------------

In [8]:
all_data[0]

,Voltage,Current,std Voltage,std Current
0,-0.000147,-50.0959,3.937120e-07,0.065890
1,-0.000144,-49.0734,5.244600e-07,0.066151
2,-0.000142,-48.0526,4.067730e-07,0.064624
3,-0.000141,-47.0351,2.150310e-07,0.064271
4,-0.000139,-46.0146,6.353970e-07,0.063555
...,...,...,...,...
96,0.006486,42.4621,2.617190e-05,0.021054
97,0.006691,43.3106,3.580150e-05,0.022281
98,0.006891,44.1577,2.950360e-05,0.020879
99,0.007007,45.0048,2.852740e-05,0.022184


In [9]:
all_data[1]

,Parameters,Unit,Value,Error
0,Je,[A/m^2],1.502360e+02,1.995400e+00
1,Jp,[A/m^2],1.767010e+00,4.205600e-03
2,Ldebye,[µm],2.489070e-04,1.046510e-06
3,Ne,[m^-3],3.050120e+15,3.560290e+13
4,Ni,[m^-3],4.981050e+15,2.690300e+13
5,Vf,[V],1.580640e+01,5.524500e-02
6,Vp,[V],2.754410e+01,2.681190e-02
7,Vpmax,[V],2.855680e+01,2.786180e-02
8,Vsat,[V],3.556960e+01,2.529790e-02
9,eedf_AvgEnergy,[eV],6.500210e+00,2.169240e-01


In [10]:
all_data[2]

,Setting,Value
0,VsatFactor,0.8
1,bUse2ndDerivativeZeroVp,0.0
2,bUseUserVp,1.0
3,correctDC,0.0
4,electronReflectVoltage,nan
5,endCrop,0.0
6,gas_temperature,300.0
7,ionmass,40.0
8,kfactor,1.34
9,nDerivativePoints,3.0


In [11]:
all_data[3]

,Extra data,Value
0,date,28.04.2023
1,deviceID,IP401-2217-D-791
2,time,15:17:30
3,Name file,name file here
4,Description,Some description here
5,Comment,some coment here


In [7]:
def df_all_data_function(root):

    tb1 = extract_parameters_from_ldf(root)
    tb2 = extract_settings_from_ldf(root)
    tb3 = extract_extradata_from_ldf(root)
    
    # este código es para los parámetros y las configuraciones (a esta ultima falta agregar unidades)
    def aux_function(aux_list):   
        value_list = aux_list[0]
        unit_list = aux_list[1]
        value_plus_unit_list =[]
        for i in range(len(unit_list)):
            if unit_list[i] != 'None':
                #value_list[i]
                value_plus_unit_list.append(value_list[i] + " " + unit_list[i])
            else:
                value_plus_unit_list[i].append(value_list[i])
        return value_plus_unit_list 

    # Codigo para unir las etiquetas de los parametros, configuraciones, e informacion extra, asi como tambien todos los valores

    label_list = aux_function(tb1)+ tb2[0] + tb3[0]
    values_list = tb1[2] + tb2[1] + tb3[1]
    data_df0 = pd.DataFrame([values_list],columns=label_list)
    data_df1 = pd.DataFrame([tb1[2]],columns=aux_function(tb1))
    data_df2 = pd.DataFrame([tb2[1]],columns=tb2[0])
    data_df3 = pd.DataFrame([tb3[1]],columns=tb3[0])
    return [data_df0, data_df1, data_df2, data_df3]

In [11]:
data__ = df_all_data_function(root)
data__[1]

,Je [A/m^2],Jp [A/m^2],Ldebye [µm],Ne [m^-3],Ni [m^-3],Vf [V],Vp [V],Vpmax [V],Vsat [V],eedf_AvgEnergy [eV],eedf_Ne [m^-3],eedf_kTe [eV],isat [mA],kTe [eV]
0,150.236,1.76701,0.000249,3.050120e+15,4.981050e+15,15.8064,27.5441,28.5568,35.5696,6.50021,2.620640e+15,4.33348,0.000147,3.41941


In [15]:
def concat_all_files_data(root_directory):

    # Lista para almacenar los archivos con la extensión .ldf
    files_ldf = []

    # Iterar sobre los archivos en la carpeta
    for file in os.listdir(root_directory):
        if file.endswith('.ldf'):
            # Agregar el nombre del archivo a la lista
            files_ldf.append(file)

    return files_ldf


In [16]:
def data_table(root_directory):

    files_ldf = concat_all_files_data(root_directory)

    df0 = df_all_data_function(root_directory + "/" + files_ldf[0])[1]
    for i in range(1,len(files_ldf)):
        root = root_directory + "/" + files_ldf[i]
        df = df_all_data_function(root)[1]
        df0 = pd.concat([df0, df], ignore_index = True)
        
    return df0
        

In [17]:
dat = data_table("../sputtering/SS-S-280423-Al-2E(-6)mbar-MR-C/Variando Potencia -10E-2mbar cte")
dat

,Je [A/m^2],Jp [A/m^2],Ldebye [µm],Ne [m^-3],Ni [m^-3],Vf [V],Vp [V],Vpmax [V],Vsat [V],eedf_AvgEnergy [eV],eedf_Ne [m^-3],eedf_kTe [eV],isat [mA],kTe [eV]
0,150.236,1.76701,0.000249,3.050120e+15,4.981050e+15,15.8064,27.5441,28.5568,35.5696,6.50021,2.620640e+15,4.33348,0.000147,3.41941
1,176.166,2.14753,0.000223,3.652580e+15,6.816740e+15,16.3443,27.3327,28.5869,37.4302,6.01669,3.224270e+15,4.01113,0.000179,3.27854
2,262.514,2.52512,0.000179,5.521490e+15,9.616780e+15,16.7663,28.1579,28.5033,32.9069,5.86439,4.880410e+15,3.90959,0.000209,3.18585
3,349.071,2.78958,0.000147,7.610110e+15,1.231100e+16,17.4374,28.6180,28.4857,31.9539,5.63282,6.620350e+15,3.75521,0.000242,2.96537
4,425.429,3.07945,0.000133,9.261890e+15,1.424410e+16,18.0343,29.3165,29.4052,31.9723,5.58673,8.075290e+15,3.72449,0.000269,2.97364
5,489.439,3.28776,0.000122,1.078340e+16,1.598940e+16,18.7819,30.1476,30.3132,32.8569,5.60581,9.262110e+15,3.73721,0.000293,2.90348
6,525.056,3.49698,0.000116,1.169930e+16,1.755660e+16,19.6181,30.7711,30.3570,33.7071,5.43339,1.013390e+16,3.62226,0.000316,2.83872
7,565.515,3.68287,0.000112,1.255190e+16,1.864010e+16,20.4568,31.6805,31.2999,33.8004,5.44769,1.090990e+16,3.63179,0.000334,2.86090


## -----------------------------------------------------------------